In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from astropy.coordinates import SkyCoord
from astropy import units as u
import glob
import os
from scipy.spatial import cKDTree
from tqdm import tqdm
from multiprocessing import Pool, Manager, freeze_support
import pickle
import json
import sys
from sklearn.metrics import accuracy_score, recall_score, precision_score, classification_report

In [1]:
master_catalog = pd.read_csv('master_updated - Copy.csv', delimiter=',')
master_catalog = master_catalog.drop('ID', axis=1)
master_catalog = master_catalog.drop('DATATABLE', axis=1)
# master_catalog = master_catalog.drop('RADEG', axis=1)
# master_catalog = master_catalog.drop('DECDEG', axis=1)
master_catalog.dropna(inplace=True)
# Create new columns for (i-g) and (g-i)
# master_catalog['i-g'] = master_catalog['i'] - master_catalog['g']
master_catalog['g-i'] = master_catalog['g'] - master_catalog['i']
# master_catalog = master_catalog.drop(master_catalog==[2 ,3 , 5, 7, 8 ,9])
# master_catalog_temp = master_catalog[master_catalog.CLASS==1]

master_catalog = master_catalog[~master_catalog['CLASS'].isin([2 , 3, 5, 7, 8 ,9])]
# class_dict = {1: 0, 4: 1, 6: 2, 2: 0, 3: 1, 9: 2, 8: 2, 7:2}
class_dict = {1: 0, 4: 1, 6: 2}

# relabel the 'CLASS' attribute using the map method
master_catalog['CLASS'] = master_catalog['CLASS'].map(class_dict)
print(master_catalog.head())
print(len(master_catalog))


NameError: name 'pd' is not defined

In [80]:
# Split the data into training and testing sets
X = master_catalog.drop('CLASS', axis=1) # Features
y = master_catalog['CLASS'] # Target variable
print((set(y)))
print(y.value_counts())
class_counts = master_catalog['CLASS'].value_counts()

# define the desired number of datapoints per class
num_datapoints = min(class_counts)

# sample the same number of datapoints from each class
master_catalog_balanced = master_catalog.groupby('CLASS').apply(lambda x: x.sample(n=num_datapoints)).reset_index(drop=True)

# print the modified dataframe
# print(master_catalog_balanced.head())
X = master_catalog_balanced.drop('CLASS', axis=1) # Features
print(X.head())
y = master_catalog_balanced['CLASS'] # Target variable
print((set(y)))
print(y.value_counts())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

{0, 1}
1    1638
0     735
Name: CLASS, dtype: int64
       RADEG     DECDEG       i       g    g-i
0  10.603958  41.209639  20.696  22.014  1.318
1  10.581171  40.872878  21.512  23.340  1.828
2  23.924083  28.820972  22.424  22.940  0.516
3  10.922904  41.407150  21.553  23.002  1.449
4  10.144979  40.443692  22.805  23.006  0.201
{0, 1}
0    735
1    735
Name: CLASS, dtype: int64


In [81]:
## create DS9 regions for 'confirmed' GCs
concat_thin_confirmed_gcs = pd.DataFrame(master_catalog_balanced)[master_catalog_balanced.CLASS==1]
radius = '10.0"'
with open('ds9_regions/confirmed.reg', 'w') as f:
    f.write('global color=green dashlist=8 3 width=2 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 '
            'fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n')
    f.write('fk5 \n')
    for n in range(0,len(concat_thin_confirmed_gcs)):
        ra = str(concat_thin_confirmed_gcs.RADEG[n:n+1].values[0])
        dec = str(concat_thin_confirmed_gcs.DECDEG[n:n+1].values[0])
        f.write('circle('+ra+','+dec+','+radius+') \n')

In [82]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [83]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=42)
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))

Logistic Regression Accuracy: 0.6530612244897959


In [84]:
from sklearn.svm import SVC

# Train and test using Support Vector Machine (SVM)
svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))


SVM Accuracy: 0.7142857142857143
              precision    recall  f1-score   support

           0       0.70      0.79      0.74        76
           1       0.74      0.63      0.68        71

    accuracy                           0.71       147
   macro avg       0.72      0.71      0.71       147
weighted avg       0.72      0.71      0.71       147



In [85]:

# Train and test using Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=200, random_state=42)
rfc.fit(X_train_scaled, y_train)
y_pred_rfc = rfc.predict(X_test_scaled)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rfc))
print("Random Forest Precision:", precision_score(y_test, y_pred_rfc, average='weighted'))
print("Random Forest Recall:", recall_score(y_test, y_pred_rfc, average='weighted'))
print(classification_report(y_test, y_pred_rfc))

Random Forest Accuracy: 0.8231292517006803
Random Forest Precision: 0.8263686426951733
Random Forest Recall: 0.8231292517006803
              precision    recall  f1-score   support

           0       0.80      0.88      0.84        76
           1       0.86      0.76      0.81        71

    accuracy                           0.82       147
   macro avg       0.83      0.82      0.82       147
weighted avg       0.83      0.82      0.82       147



In [86]:
# from xgboost import XGBClassifier
# # Train and test using XGBoost
# xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
# xgb.fit(X_train_scaled, y_train)
# y_pred_xgb = xgb.predict(X_test_scaled)
# print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))

In [87]:
# one_hot = pd.get_dummies(master_catalog['CLASS'], prefix='CLASS')
#
# # concatenate the one-hot vector with the original dataframe
# master_catalog = pd.concat([master_catalog, one_hot], axis=1)
#
# # drop the original 'CLASS' attribute
# master_catalog = master_catalog.drop('CLASS', axis=1)
#
# # print the modified dataframe
# print(master_catalog)

In [88]:
import numpy as np
values = np.array(master_catalog_balanced['CLASS'], dtype='int')
n_values = 2
print(n_values)
onehot = np.eye(n_values)[values]
print(onehot.shape)
X_train, X_test, y_train, y_test = train_test_split(X, onehot, test_size=0.1, random_state=42)

2
(1470, 2)


In [89]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
# Define the neural network architecture
model = Sequential()
model.add(Dense(15, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(2, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=4, verbose=1)


Epoch 1/20
331/331 [==============================] - 1s 964us/step - loss: 2.9191 - accuracy: 0.5737
Epoch 2/20
331/331 [==============================] - 0s 949us/step - loss: 0.7163 - accuracy: 0.6047
Epoch 3/20
331/331 [==============================] - 0s 867us/step - loss: 0.7026 - accuracy: 0.5949
Epoch 4/20
331/331 [==============================] - 0s 867us/step - loss: 0.6729 - accuracy: 0.6213
Epoch 5/20
331/331 [==============================] - 0s 864us/step - loss: 0.6650 - accuracy: 0.6017
Epoch 6/20
331/331 [==============================] - 0s 876us/step - loss: 0.6574 - accuracy: 0.6213
Epoch 7/20
331/331 [==============================] - 0s 843us/step - loss: 0.6606 - accuracy: 0.6115
Epoch 8/20
331/331 [==============================] - 0s 861us/step - loss: 0.6876 - accuracy: 0.6070
Epoch 9/20
331/331 [==============================] - 0s 867us/step - loss: 0.6663 - accuracy: 0.6145
Epoch 10/20
331/331 [==============================] - 0s 907us/step - loss: 0.643

In [90]:
# Test the model
# y_pred_nn = model.predict(X_test)
# print(y_pred_nn)
# y_pred_nn_classes = (y_pred_nn.argmax()).astype(int)
pred = model.predict(X_test)
prediction = np.argmax(pred, axis=-1)
y_true = np.argmax(y_test, axis=-1)
print((pd.DataFrame(prediction)).value_counts())
print((pd.DataFrame(y_true)).value_counts())
print("Neural Network Accuracy:", accuracy_score(y_true, prediction))
print("Classification Report: ", classification_report(y_true, prediction))

5/5 [==============================] - 0s 1ms/step
0    129
1     18
dtype: int64
0    76
1    71
dtype: int64
Neural Network Accuracy: 0.5578231292517006
Classification Report:                precision    recall  f1-score   support

           0       0.54      0.92      0.68        76
           1       0.67      0.17      0.27        71

    accuracy                           0.56       147
   macro avg       0.60      0.55      0.48       147
weighted avg       0.60      0.56      0.48       147



In [91]:
# Create an instance of the Random Forest Classifier
rfc = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
rfc.fit(X_train, y_train)
with open('rfc_model.pkl', 'wb') as f:
    pickle.dump(rfc, f)

In [92]:
with open('rfc_model.pkl', 'rb') as f:
    rfc = pickle.load(f)

In [93]:
# Use the trained model to make predictions on the test data
y_pred = rfc.predict(X_test)

# Evaluate the performance of the model
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.7891156462585034


In [94]:
#Importing essential libraries
import matplotlib.pyplot as plt
from statistics import mean
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
# from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier

#Build SRF model
SRF = RandomForestClassifier(n_estimators=150, random_state=0)
#Create Stratified K-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scoring = ('f1', 'recall', 'precision')
#Evaluate SRF model
# scores = cross_validate(SRF, X, y, scoring=scoring, cv=cv)
# #Get average evaluation metrics
# print('Mean f1: %.3f' % mean(scores['test_f1']))
# print('Mean recall: %.3f' % mean(scores['test_recall']))
# print('Mean precision: %.3f' % mean(scores['test_precision']))

#Randomly spilt dataset to test and train set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)
#Train SRF
SRF.fit(X_train, y_train)
#SRF prediction result
y_pred = SRF.predict(X_test)
#Create confusion matrix
# fig = plot_confusion_matrix(SRF, X_test, y_test, display_labels=['Will Not Buy', 'Will Buy'], cmap='Greens')
# plt.title('Standard Random Forest Confusion Matrix')
# plt.show()

In [95]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7653061224489796
              precision    recall  f1-score   support

           0       0.77      0.75      0.76       147
           1       0.76      0.78      0.77       147

    accuracy                           0.77       294
   macro avg       0.77      0.77      0.77       294
weighted avg       0.77      0.77      0.77       294



TEST DATA

In [96]:
with open('rfc_model.pkl', 'rb') as f:
    rfc = pickle.load(f)

In [97]:
m001p = pd.read_csv('Cleaned/cm233p.ascd', delimiter=' ')
new_columns = {'Dec': 'DECDEG', 'RA': 'RADEG'}
m001p = m001p.rename(columns=new_columns)
col = m001p.pop('g')
m001p.insert(3, 'g', col)
# m001p.iloc[:, [2, 3]] = m001p.iloc[:, [3, 2]]
m001p['g-i'] = m001p['g'] - m001p['i']
# ra_deg = float(row['RAh']) * 15 + float(row['RAm']) * 0.25 + float(row['RAs']) * 0.00416667
# dec_deg = float(row['DEd']) + (float(row['DEm'])/60) + (float(row['DEs'])/3600)
catalog_coord = SkyCoord(ra=m001p['RADEG'], dec=m001p['DECDEG'], unit=(u.hourangle, u.deg))
m001p['RADEG'] = catalog_coord.ra.degree
m001p['DECDEG'] = catalog_coord.dec.degree
print(m001p.head())

       RADEG     DECDEG       i       g    g-i
0  10.078462  40.667978  18.543  18.514 -0.029
1  10.071896  40.651283  18.481  18.578  0.097
2  10.012758  40.566219  18.167  18.631  0.464
3  10.064004  40.462811  18.353  18.637  0.284
4  10.060642  40.622356  18.760  18.627 -0.133


In [98]:
# y_values_prob = np.array(rfc.predict_proba(m001p))[0]
# y_values = np.array(rfc.predict(m001p))[0]
# print((y_values.shape))
# idx = y_values[:, 0].argsort()[::-1]
# sorted_arr = y_values[idx]
# print(sorted_arr)
# print(y_values)
# print(y_values[0][0])

In [99]:
# probs = np.array(rfc.predict_proba(m001p))[:, 0]
#
# # Sort the probabilities in descending order
# sorted_probs_idx = probs.argsort()[::-1]
# sorted_probs = probs[sorted_probs_idx]
#
# print(sorted_probs)
# # Get the corresponding rows from the original DataFrame
# sorted_rows = m001p.iloc[sorted_probs_idx].values.flatten()
#
# # Combine the rows and probabilities into a new DataFrame
# result = pd.concat([sorted_rows, pd.DataFrame(sorted_probs, columns=['Probability of class 0'])], axis=1)
#
# # Display the result
# print(result)

In [100]:
# import numpy as np
#
# # Assuming 'rfc' is your RandomForestClassifier instance and 'm001p' is your input data
#
# # Get the predicted probabilities for each class
# predicted_probabilities = np.array(rfc.predict_proba(m001p))
# print(predicted_probabilities[0])
# # Extract the probabilities for class 0 (first column)
# class_0_probabilities = predicted_probabilities[1, :, 1]
#
# # Get the indices of the sorted probabilities in descending order
# sorted_indices = np.argsort(class_0_probabilities)[::-1]
# print(sorted_indices)
# # Sort the probabilities using the sorted indices
# sorted_class_0_probabilities = class_0_probabilities[sorted_indices]
#
# # Print the ordered list of best matches for class 0
# print(sorted_class_0_probabilities)
# # Assuming 'm001p' is a 2D numpy array or a pandas DataFrame
#
# # Get the index of the best match for class 0
# best_match_data = []
# for indic in sorted_indices:
#     best_match_index = indic
#
#     # Get the data corresponding to the best match
#     best_match_data.append(m001p.iloc[best_match_index])
#
# # Print the best match data
# print(best_match_data)

In [101]:
import numpy as np
SRF = RandomForestClassifier(n_estimators=150, random_state=0)
#Create Stratified K-fold cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scoring = ('f1', 'recall', 'precision')
X_train, X_test, _, _ = train_test_split(X, y, test_size=0, stratify=y)
#Train SRF
SRF.fit(X_train, y_train)
#SRF prediction result
class_labels = SRF.predict(m001p)
print(class_labels)
# class_labels = rfc.predict(m001p)

# get the indices of the elements that are predicted to be class 0
class_0_indices = np.where(class_labels == 1)[0]
class_0_indices = np.unique(class_0_indices)
# get the data from `m001p` corresponding to the class 0 predictions
class_0_data = m001p.iloc[class_0_indices, :]
print(class_0_data.head())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

m001p_scaled = scaler.transform(m001p)
# Assuming 'rfc' is your RandomForestClassifier instance and 'm001p' is your input data

# Get the predicted probabilities for each class

#Evaluate SRF model
# scores = cross_validate(SRF, X, y, scoring=scoring, cv=cv)
# #Get average evaluation metrics
# print('Mean f1: %.3f' % mean(scores['test_f1']))
# print('Mean recall: %.3f' % mean(scores['test_recall']))
# print('Mean precision: %.3f' % mean(scores['test_precision']))

#Randomly spilt dataset to test and train set


svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
print(classification_report(y_test, y_pred_svm))
svm_class_labels = svm.predict(m001p_scaled)
print(set(svm_class_labels))
# get the indices of the elements that are predicted to be class 0
svm_indices = np.where(svm_class_labels == 0)[0]
svm_indices = np.unique(svm_indices)
# get the data from `m001p` corresponding to the class 0 predictions
svm_data = m001p.iloc[svm_indices, :]
print(svm_data.head())


ValueError: test_size=0 should be either positive and smaller than the number of samples 1470 or a float in the (0, 1) range

In [ ]:
print(len(svm_data))
print(len(class_0_data))
merged_data = pd.merge(svm_data, class_0_data, how='inner')

# create a new dataframe that contains only the rows that match between `svm_data` and `class_0_data`
matched_data = svm_data[svm_data.isin(merged_data.to_dict(orient='list')).all(1)]

print(matched_data.head())
print(len(matched_data))

In [ ]:
## create DS9 regions for 'confirmed' GCs
concat_thin_confirmed_gcs = pd.DataFrame(matched_data)
radius = '10.0"'
with open('ds9_regions/m233_cut.reg', 'w') as f:
    f.write('global color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 \n')
    f.write('fk5 \n')
    for n in range(0,len(concat_thin_confirmed_gcs)):
        ra = str(concat_thin_confirmed_gcs.RADEG[n:n+1].values[0])
        dec = str(concat_thin_confirmed_gcs.DECDEG[n:n+1].values[0])
        f.write('circle('+ra+','+dec+','+radius+') \n')